## Отчет по лабороторной работе №1

Для начала привожу листинг с некоторыми комментариями

In [5]:
import string
import math
from itertools import groupby
from itertools import chain
from operator import itemgetter
from numpy import arange

import nltk
from nltk import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

default_tokenize_func = word_tokenize
default_lemmatizer = WordNetLemmatizer()
default_stemmer = SnowballStemmer("english")
default_stop_words = set(nltk.corpus.stopwords.words('english'))

# код из eval.py, завернутый в функцию
def eval_metrics(groundtruth_file, answer_file):
    q2reld = {} 
    for line in open(groundtruth_file):
        qid, did = [int(x) for x in line.split()]
        if qid not in q2reld.keys():
            q2reld[qid] = set()
        q2reld[qid].add(did)        

    q2retrd = {}
    for line in open(answer_file):
        qid, did = [int(x) for x in line.split()]
        if qid not in q2retrd.keys():
            q2retrd[qid] = []
        q2retrd[qid].append(did)               

    N = len(q2retrd.keys())
    precision = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2retrd[q]) for q in q2retrd.keys()]) / N
    recall = sum([len(q2reld[q].intersection(q2retrd[q]))*1.0/len(q2reld[q]) for q in q2retrd.keys()]) / N
#    print("mean precision: {}\nmean recall: {}\nmean F-measure: {}"\
#          .format(precision, recall, 2*precision*recall/(precision+recall)))

    # MAP@10
    import numpy as np

    MAP = 0.0
    for q in q2retrd.keys():
        n_results = min(10, len(q2retrd[q]))
        avep = np.zeros(n_results)
        for i in range(n_results):
            avep[i:] += q2retrd[q][i] in q2reld[q]
            avep[i] *= (q2retrd[q][i] in q2reld[q]) / (i+1.0)
        MAP += sum(avep) / min(n_results, len(q2reld[q]))
#    print("MAP@10: {}".format(MAP/N))

    return precision, recall, 2*precision*recall/(precision+recall), MAP/N

# функция для парсинга файла
# вырезает куски соответствующие определенной секции в файле и возвращает сам кусок и присвоенный ему идентификатор
def parse_file(file, label='W'):
    section_labels = {'I', 'T', 'A', 'B', 'W'}

    section_found = False
    id = 1

    with open(file, 'r') as f:

        document = ''

        for line in f.readlines():

            stipped_line = line.strip()

            if len(stipped_line) == 0:
                continue

            if stipped_line.startswith('.' + label):
                section_found = True
            elif len(stipped_line) >= 2 and stipped_line[0] == '.' and stipped_line[1] in section_labels:
                if section_found:
                    yield (id, document.strip())
                    document = ''
                    id += 1
                    section_found = False

            elif section_found:
                document += ' ' + stipped_line

        if section_found:
            yield (id, document.strip())

# функция нормализации текста
# бьет текст на токены, удаляет стоп-слова(если передан список слов), применяет лемматизатор и стеммер (переданы)
# возвращает токен с его частотой
def normalize_text(text,
                   tokenize_func=default_tokenize_func,
                   lemmatizer=default_lemmatizer,
                   stemmer=default_stemmer,
                   stop_words=default_stop_words):
    tokens = list()

    for token in tokenize_func(text):
        if token in string.punctuation:
            continue

        if token in stop_words:
            continue

        if lemmatizer:
            token = lemmatizer.lemmatize(token)

        if stemmer:
            token = stemmer.stem(token)

        tokens.append(token)

    tokens.sort()

    return [(key, len(list(group))) for key, group in groupby(tokens)]

# расчет RSV по заданным параметрам (соответствует пункту 3 в задании)
def calc_rsv_default(N, Nt, ftd, ftq, Ld, L, b, k1, k2):
    idf = calc_idf_default(N, Nt)
    tf = ftd * (k1 + 1) / (k1 * ((1 - b) + b * Ld / L) + ftd)
    return idf * tf

# вспомогательная функция для расчета IDF
def calc_idf_default(N, Nt):
    return math.log(1 + (N - Nt + 0.5) / (Nt + 0.5))

# инвертированный индекс с возможностью добавления документов по одному и поиску
class InvertedIndex:
    def __init__(self, lemmatizer=default_lemmatizer, stemmer=default_stemmer):
        self.lemmatizer = lemmatizer
        self.stemmer = stemmer
        self.documents_total_length = 0
        self.documents = dict()
        self.index = defaultdict(list)

        pass

    # добавление документа
    def add_document(self, doc_id, document):
        self.documents_total_length += len(document)
        self.documents[doc_id] = document

        # todo maybe sort after add
        for token in normalize_text(document, lemmatizer=self.lemmatizer, stemmer=self.stemmer):
            self.index[token[0]].append((doc_id, token[1]))

        pass

    # поиск
    def search_okapi_bm25(self, query, b, k1, k2, rsvfunc, norm_rsv, limit=10):
        result = defaultdict(int)

        N = len(self.documents)
        L = self.documents_total_length / len(self.documents)

        idf_sums = defaultdict(int)
        
        for token_tuple in normalize_text(query, lemmatizer=self.lemmatizer, stemmer=self.stemmer):
            ftq = token_tuple[1]
            token_docs = self.index[token_tuple[0]]
            Nt = len(token_docs)
            for doc_tuple in token_docs:
                doc_id = doc_tuple[0]
                ftd = doc_tuple[1]
                Ld = len(self.documents[doc_id])

                result[doc_id] += rsvfunc(N, Nt, ftd, ftq, Ld, L, b, k1, k2)
                    
                if norm_rsv :
                    idf_sums[doc_id] += calc_idf_default(N, Nt)

        result = list(result.items())

        if norm_rsv:
            result = [(x[0], x[1] / idf_sums[x[0]]) for x in result]

        result = sorted(result, key=itemgetter(1), reverse=True)
        #print(result[:10])
        return [x[0] for x in result][:limit]

    # размер словаря
    def get_dist_size(self):
        return len(self.index)
    
    # средняя длина posting-листа
    def get_average_postings_list_length(self):
        l = [len(p) for p in self.index.values()]
        return sum(l) / len(l)

    # максимальная длина posting-листа
    def get_max_postings_list_length(self):
        l = [len(p) for p in self.index.values()]
        return max(l)
    
    def print_statistics(self):
        print("\tStatistics:\n\tDictionary size = {}\n\tAverage postings list length = {}\n\tMax postings list length = {}"
              .format(self.get_dist_size(), self.get_average_postings_list_length(), self.get_max_postings_list_length()))



In [2]:
text = """
experimental investigation of the aerodynamics of a
wing in a slipstream .
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with
supporting evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or
boundary-layer-control effect .  the integrated remaining lift
increment, after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment ."""
print(normalize_text(text))

[('/destalling/', 1), ('aerodynam', 1), ('agre', 1), ('angl', 1), ('attack', 1), ('basi', 1), ('boundary-layer-control', 1), ('compar', 1), ('configur', 1), ('curv', 1), ('destal', 2), ('determin', 1), ('differ', 3), ('distribut', 1), ('due', 2), ('effect', 2), ('empir', 1), ('evalu', 2), ('evid', 1), ('experi', 1), ('experiment', 2), ('flow', 1), ('found', 1), ('free', 1), ('increas', 1), ('increment', 2), ('integr', 1), ('intend', 1), ('investig', 1), ('lift', 4), ('load', 1), ('made', 2), ('order', 1), ('part', 2), ('potenti', 1), ('problem', 1), ('produc', 1), ('propel', 1), ('ratio', 1), ('remain', 1), ('result', 1), ('show', 1), ('slipstream', 5), ('span', 1), ('spanwis', 1), ('specif', 1), ('stream', 1), ('studi', 1), ('substanti', 1), ('subtract', 1), ('support', 1), ('theoret', 1), ('theori', 1), ('togeth', 1), ('treatment', 1), ('veloc', 1), ('well', 1), ('wing', 3)]


Заполним инвертированный индекс используя данные тексты.

In [6]:
# индекс по заголовкам (с использованием стеммера)
indexTS = InvertedIndex()
# индекс по заголовкам (с использованием лемматизатора)
indexTL = InvertedIndex(lemmatizer=WordNetLemmatizer(), stemmer=None)
# индекс по всему abstract (с использованием стеммера)
indexWS = InvertedIndex()
# индекс по всему abstract (с использованием лемматизатора)
indexWL = InvertedIndex(lemmatizer=WordNetLemmatizer(), stemmer=None)
for doc_tuple in parse_file('cran.all.1400', 'T'):
    indexTS.add_document(doc_tuple[0], doc_tuple[1])
    indexTL.add_document(doc_tuple[0], doc_tuple[1])
for doc_tuple in parse_file('cran.all.1400', 'W'):
    indexWS.add_document(doc_tuple[0], doc_tuple[1])
    indexWL.add_document(doc_tuple[0], doc_tuple[1])
    
words = list(indexTS.index.keys())
words = sorted(words)
#print(words)
#for key in words:
#    print("", key, ": ", indexTS.index[key], "\n")

Проведем поиск по каждому из индексов и рассчитаем показатели качества (вместе с этим, распечатаем статистику по каждому из индексов)

In [7]:
def parse_search_eval(index, b=0.75, k1=1.2, k2=None, rsvfunc=calc_rsv_default, norm_rsv=False):
    with open('answer', 'w') as output:
        for query_tuple in parse_file('cran.qry'):
            query_id = query_tuple[0]
            for doc_id in index.search_okapi_bm25(query_tuple[1], b=b, k1=k1, k2=k2, rsvfunc=rsvfunc, norm_rsv=norm_rsv):
                output.write("{} {}\n".format(query_id, doc_id))

    return eval_metrics('qrel_clean', 'answer')

precision, recall, fmeasure, map10 = parse_search_eval(indexTS)
print("Index For Titles (stemmer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
indexTS.print_statistics()

# precision, recall, fmeasure, map10 = parse_search_eval(indexTL)
# print("Index For Titles (lemmatizer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
# indexTL.print_statistics()

# precision, recall, fmeasure, map10 = parse_search_eval(indexWS)
# print("Index For Abstracts (stemmer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
# indexWS.print_statistics()

# precision, recall, fmeasure, map10 = parse_search_eval(indexWL)
# print("Index For Abstracts (lemmatizer)\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))
# indexWL.print_statistics()

Index For Titles (stemmer)
	Precision = 0.4
	Recall = 0.13793103448275862
	F-measure = 0.20512820512820515
	MAP@10 = 0.2671428571428572
	Statistics:
	Dictionary size = 1529
	Average postings list length = 7.087638979725311
	Max postings list length = 358


Как и ожидалось, все показатели оказались лучше для индекса, построенного по abstracts, чем по titiles, так как увеличивается вероятностью встретить одинаковые термы и в запросе, и в тексте. Если сравнивать индексы, при построении и поиске по которым использовался стеммер с теми, где использовался лемматизатор, видно, что результаты приблизительно одинаковые, поэтому для дальнейших экспериментов решено было использовать стеммер (он быстрее работает). 

In [8]:
index = indexWS

default_best_b = 0.9
default_best_k1 = 1.3

Далее, используя Grid Search, найдем оптимальные параметры поиска (вычисление занимает несколько минут, поэтому строки закомментированы)

In [9]:
best_by_fmeasure = (0, 0, 0, 0, 0, 0)
best_by_map10 = (0, 0, 0, 0, 0, 0)

for k1 in arange(1.2, 2.1, .1):
    for b in arange(.0, 1.1, .1):
        precision, recall, fmeasure, map10 = parse_search_eval(index, b=b, k1=k1)
        if fmeasure > best_by_fmeasure[2]:
            best_by_fmeasure = (precision, recall, fmeasure, map10, b, k1)

        if map10 > best_by_map10[3]:
            best_by_map10 = (precision, recall, fmeasure, map10, b, k1)


In [10]:
print("Best f-measure = {} at b = {}, k1 = {}".format(best_by_fmeasure[2], best_by_fmeasure[4], best_by_fmeasure[5]))
print("Best map10 = {} at b = {}, k1 = {}".format(best_by_map10[3], best_by_map10[4], best_by_map10[5]))

default_best_b = best_by_fmeasure[4]
default_beft_k1 = best_by_fmeasure[5]

Best f-measure = 0.25641025641025644 at b = 0.7000000000000001, k1 = 1.9000000000000006
Best map10 = 0.38769841269841265 at b = 0.7000000000000001, k1 = 2.000000000000001


Я думаю, что оптимальный для f-меры параметр b получился равным 0.9, потому что тексты в среднем значительно отличаются по длине. А параметр k1 равным 1.3, скорее всего потому что частота всех термов запросов в документах в среднем одинаковая.

Попробуем заменить функцию вычисления IDF-составляющей, используя при этом значения b и k1, для которых было получено лучшее значение f-меры.

In [11]:
def calc_rsv_custom_idf(N, Nt, ftd, ftq, Ld, L, b, k1, k2):
    idf = math.log(N / Nt)
    tf = ftd * (k1 + 1) / (k1 * ((1 - b) + b * Ld / L) + ftd)
    return idf * tf

precision, recall, fmeasure, map10 = parse_search_eval(index, b=default_best_b, k1=default_beft_k1, rsvfunc=calc_rsv_custom_idf)
print("Results:\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))

Results:
	Precision = 0.5
	Recall = 0.1724137931034483
	F-measure = 0.25641025641025644
	MAP@10 = 0.375


Как видно, данное изменение не привело к значительному улучшению, кроме того, я думаю, что при большом количестве документов привело бы к ухудшению значения f-меры.

Далее, попробуем нормализовать RSV документа на сумму IDF термов запроса, которые в него входят.

In [13]:
precision, recall, fmeasure, map10 = parse_search_eval(index, b=default_best_b, k1=default_beft_k1, norm_rsv=True)
print("Results:\n\tPrecision = {}\n\tRecall = {}\n\tF-measure = {}\n\tMAP@10 = {}".format(precision, recall, fmeasure, map10))

ZeroDivisionError: float division by zero

Нормализация несколько ухудшила значение f-меры и сильно ухудшила значение MAP@10. Благодаря такому приему, должно было уменьшиться влияние частовстречающихся слов на полученный RSV, однако, скорее всего из-за того что стоп-слова были отфильтрованы, результаты только ухудшились.

Попробуем как предлагается добавить параметр k2, используя до этого полученные лучшие b и k1.

In [9]:
def calc_rsv_general(N, Nt, ftd, ftq, Ld, L, b, k1, k2):
    idf = calc_idf_default(N, Nt)
    tftd = ftd * (k1 + 1) / (k1 * ((1 - b) + b * Ld / L) + ftd)
    tftq = ftq * (k2 + 1) / (k2 + ftq)
    return idf * tftd * tftq

best_by_fmeasure_general = (0, 0, 0, 0, 0, 0, 0)
best_by_map10_general = (0, 0, 0, 0, 0, 0, 0)

for k2 in (0, 1, 5, 10, 20, 50, 100, 200, 500, 1000):
    precision, recall, fmeasure, map10 = parse_search_eval(index, b=default_best_b, k1=default_beft_k1, k2=k2, rsvfunc=calc_rsv_general)
    if fmeasure > best_by_fmeasure_general[2]:
        best_by_fmeasure_general = (precision, recall, fmeasure, map10, default_best_b, default_beft_k1, k2)

    if map10 > best_by_map10_general[3]:
        best_by_map10_general = (precision, recall, fmeasure, map10, default_best_b, default_beft_k1, k2)

In [10]:
print("Best f-measure = {} at b = {}, k1 = {}, k2 = {}"
      .format(best_by_fmeasure_general[2], best_by_fmeasure_general[4], best_by_fmeasure_general[5], best_by_fmeasure_general[6]))
print("Best map10 = {} at b = {}, k1 = {}, k2 = {}"
      .format(best_by_map10_general[3], best_by_map10_general[4], best_by_map10_general[5], best_by_map10_general[6]))

Best f-measure = 0.2421726419438806 at b = 0.9, k1 = 1.3, k2 = 0
Best map10 = 0.20365195193303656 at b = 0.9, k1 = 1.3, k2 = 1


Получилось, что при k2 = 0, мы достигли лучшего значения f-меры. Это можно объяснить тем, что скорее всего термы в запросах встречались один - два раза и это не повлекло за собой значительного изменения RSV.

Таким образом, лучшее полученное значение f-меры равно 0.2427002488648061 (при подборе параметров и использовании более простой формулы IDF), а лучшее значение MAP@10 равно 0.20379489585957844 (при подборе параметров). Ни одна из предложенных оптимизаций не смогла на данном наборе документов и запросов значительно улучшить результаты поиска. Полученные результаты оказались не слишком хороши в первую очередь потому что модель BM25 не учитывает множество факторов, которые учитывают более новые модели. А также из-за небольшого объема коллекции и малой длины текстов.